![](images/locust.png)

One of the most popular performance testing frameworks for python is https://locust.io.

The code below and a sample implementation of a performance test on {JSON} Placeholder API Server.

```python
import random
from locust import HttpUser, between, task
from locust.env import Environment
from faker import Faker

class APIPerfTest(HttpUser):
    wait_time = between(5, 15)
    
    def on_start(self):
        """
        This function is executed before the performance test
        """
        self.user_ids = []
        self.faker = Faker()
    
    def create_profile(self):
        """
        Generate random profile
        """
        return {
            "name": self.faker.name(),
            "username": self.faker.email().split('@')[0],
            "email": self.faker.company_email(),
            "phone": self.faker.phone_number(),
            "website": self.faker.domain_name(),
            "address": {
                "street": self.faker.street_address(),
                "suite": self.faker.building_number(),
                "city": self.faker.city(),
                "zipcode": self.faker.postcode()
            },
            "company": {
                "name": self.faker.company(),
                "catchPhrase": self.faker.catch_phrase(),
                "bs": self.faker.bs()
            }
        }
    
    @task
    def get_users(self):
        """
        Fetch users
        """
        self.client.get("/users")

    @task
    def add_user(self):
        """
        Add a new user
        """
        r = self.client.post("/users", self.create_profile())
        self.user_ids.append(r.json()['id'])

    @task
    def get_user(self):
        """
        Fetch a random user
        """
        if self.user_ids:
            id = random.choice(self.user_ids)
            self.client.get("/users/{}".format(id))

    @task
    def update_user(self):
        """
        Update a random user with new profile
        """
        if self.user_ids:
            id = random.choice(self.user_ids)
            self.client.put("/users/{}".format(id), self.create_profile())

    @task
    def delete_user(self):
        """
        Delete a random user
        """
        if self.user_ids:
            id =  random.choice(self.user_ids)
            self.client.delete("/users/{}".format(id))

```

Unfortunately this can't be executed from jupyter. In order to run this make sure that:

* You have the json-server running in the background. Execute it with ```json-server --watch db.json```
* Run with ```locust -f src/performance-test.py```
* And then open http://127.0.0.1:8089 to view the Web UI
* On the Web UI. Specify the following parameters (feel free to change the # users and spawn rate):
![](images/locust_config.png)
* The performance test will then start in the background. You may explore the charts page.
![](images/locust_charts.png)

Locust can also run in distributed mode in case you need to setup a bigger swarm: https://docs.locust.io/en/stable/running-locust-distributed.html